## 2. `transformer_encoder_block.ipynb`

Implements a simplified **Transformer Encoder Block** using **PyTorch**.

### 🔧 Components

* **Multi-head self-attention** (`nn.MultiheadAttention`)
* **Feed-forward network**
  `Linear → ReLU → Linear`
* **Residual (skip) connections**
* **Layer normalization** (Add & Norm)

### ⚙️ Configuration

* `d_model = 128`
* `num_heads = 8`

### 🧪 Included Test

* **Batch size:** 32 sentences
* **Sequence length:** 10 tokens
* **Embedding dimension:** 128

It prints the output shape to verify correctness.


In [1]:

from typing import Optional

import torch
from torch import nn

In [2]:

class TransformerEncoderBlock(nn.Module):
    """
    A simplified Transformer encoder block.

    Parameters
    ----------
    d_model : int
        Dimensionality of input embeddings (and the attention model).
    num_heads : int
        Number of attention heads in the multi-head attention layer.
    dim_feedforward : int, optional
        Hidden dimension of the feed-forward network. Typically 2-4x d_model.
    dropout : float, optional
        Dropout probability applied after attention and feed-forward layers.

    Inputs
    ------
    x : torch.Tensor
        Input tensor of shape (batch_size, seq_len, d_model).

    Returns
    -------
    torch.Tensor
        Output tensor of shape (batch_size, seq_len, d_model).
    """

    def __init__(
        self,
        d_model: int = 128,
        num_heads: int = 8,
        dim_feedforward: int = 512,
        dropout: float = 0.1,
    ) -> None:
        super().__init__()

        # Multi-head self-attention layer.
        # Using batch_first=True means inputs are (batch, seq, d_model)
        self.self_attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=num_heads,
            dropout=dropout,
            batch_first=True,
        )

        # Feed-forward network: Linear -> ReLU -> Linear
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, d_model),
        )

        # Layer normalization layers for "Add & Norm"
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Dropout layers applied after attention and feed-forward sublayers
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(
        self,
        x: torch.Tensor,
        src_key_padding_mask: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:
        """
        Forward pass of the Transformer encoder block.

        Parameters
        ----------
        x : torch.Tensor
            Input tensor of shape (batch_size, seq_len, d_model).
        src_key_padding_mask : torch.Tensor, optional
            Boolean mask of shape (batch_size, seq_len) where True indicates
            positions that should be ignored (e.g., padding tokens).

        Returns
        -------
        torch.Tensor
            Tensor of shape (batch_size, seq_len, d_model) after applying
            self-attention, feed-forward network, and Add & Norm layers.
        """
        # ---- 1. Multi-head self-attention sublayer ----
        # For self-attention: query = key = value = x
        attn_output, _ = self.self_attn(
            query=x,
            key=x,
            value=x,
            key_padding_mask=src_key_padding_mask,
            need_weights=False,  # We only need the transformed output here
        )

        # Apply dropout to attention output
        attn_output = self.dropout1(attn_output)

        # Residual connection: add the original input (skip connection)
        x = x + attn_output

        # Layer normalization (first Add & Norm)
        x = self.norm1(x)

        # ---- 2. Position-wise feed-forward sublayer ----
        ffn_output = self.ffn(x)

        # Dropout after feed-forward
        ffn_output = self.dropout2(ffn_output)

        # Second residual connection
        x = x + ffn_output

        # Second layer normalization (second Add & Norm)
        x = self.norm2(x)

        return x



In [3]:
# Quick shape test to satisfy the assignment's sub-task (c).

# Parameters from the problem statement
batch_size = 32     # number of sentences
seq_len = 10        # number of tokens per sentence
d_model = 128       # embedding dimension (must match the block)

# Create random input: (batch_size, seq_len, d_model)
x = torch.randn(batch_size, seq_len, d_model)

# Initialize our encoder block
encoder_block = TransformerEncoderBlock(d_model=d_model, num_heads=8)

# Forward pass
output = encoder_block(x)

print("Input shape :", x.shape)       # torch.Size([32, 10, 128])
print("Output shape:", output.shape)  # torch.Size([32, 10, 128])

Input shape : torch.Size([32, 10, 128])
Output shape: torch.Size([32, 10, 128])
